In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import tqdm.notebook as tqdm

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
import pandas as pd
df = pd.read_csv('Data/Hindi/hin.txt',sep='\t', header=None)
df.columns =['english', "hindi", "attribution"]

In [4]:
df.head()

,english,hindi,attribution
0,Wow!,वाह!,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
1,Duck!,झुको!,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Duck!,बतख़!,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Help!,बचाओ!,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
4,Jump.,उछलो.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...


In [5]:
df.drop("attribution", axis=1, inplace=True)

In [6]:
english_hindi_dataset = []

In [7]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [8]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    #s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [9]:
def readLangs(lang1, lang2, df, reverse=False):
    '''
    Read lines, from text file.
    lang1 - laguage as input,
    lang2 - output language,
    df - dataframe
    reverse - to reverse the languages as input and output.
    '''
    print("Reading lines...")

    # # Read the file and split into lines
    # lines = open('Data/Hindi/hin.txt'.format(lang1, lang2), encoding='utf-8').\
    #     read().strip().split('\n')
    # Split every line into pairs and normalize
    
    # pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    # pairs = [[z.split("\t") for z in normalizeString(f"{x}\t{y}")] for x, y in zip(df[lang1], df[lang2])]
    pairs = [[normalizeString(df[lang1][i]), normalizeString(df[lang2][i])] for i in df.index]
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [10]:
MAX_LENGTH = 15 #max length of words in a sentence.

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH #and \
        #p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [11]:
def prepareData(lang1, lang2, df, reverse=False):
    '''
    prepare class objects for the languages.
    '''
    input_lang, output_lang, pairs = readLangs(lang1, lang2, df, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('english', 'hindi', df, False)
print(random.choice(pairs))

Reading lines...
Read 2980 sentence pairs
Trimmed to 2932 sentence pairs
Counting words...
Counted words:
english 2474
hindi 3062
['i know tom is an electrical engineer .', 'मुझे पता है कि टॉम एक इलेक्ट्रिकल इंजीनियर है।']


In [12]:
df.head()

,english,hindi
0,Wow!,वाह!
1,Duck!,झुको!
2,Duck!,बतख़!
3,Help!,बचाओ!
4,Jump.,उछलो.


In [25]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)


class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output))
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)


In [15]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        self.softmax = nn.LogSoftmax(dim=2)  # Update the dimension for LogSoftmax

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input)  # Remove view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded, hidden), 2)), dim=2)  # Update the dimension for concatenation
        attn_applied = torch.bmm(attn_weights,  # Remove unsqueeze(0)
                                 encoder_outputs)

        output = torch.cat((embedded, attn_applied), 2)  # Update the dimension for concatenation
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = self.softmax(self.out(output))  # Update the dimension for LogSoftmax
        return output, hidden, attn_weights

    def initHidden(self, batch_size):  # Add batch_size argument
        return torch.zeros(1, batch_size, self.hidden_size, device=device)  # Update size of hidden tensor


In [16]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [31]:
teacher_forcing_ratio = 0
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden(input_tensor.size(1))  # Initialize hidden state for batch input
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0]

    decoder_input = torch.tensor([[SOS_token]] * input_tensor.size(1), device=device)  # Modify for batch input

    decoder_hidden = encoder_hidden.unsqueeze(0)  # Add an additional dimension for batch input

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, _ = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: Use the decoder's own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, _ = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


In [18]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


In [19]:
from torch.utils.data import DataLoader

def pad_sequence(sequence, max_length):
    # Pad sequence with zeros to match the max_length
    padded_sequence = torch.zeros((max_length, *sequence[0].shape), dtype=sequence[0].dtype)
    for i, tensor in enumerate(sequence):
        padded_sequence[i] = tensor
    return padded_sequence

def trainIters(encoder, decoder, n_iters, batch_size=1, print_every=1000, plot_every=100, learning_rate=0.01):
    global pairs  # Access the global pairs variable
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    # Determine the maximum sequence length in the training pairs
    max_length = max(max(len(pair[0]), len(pair[1])) for pair in training_pairs)

    # Pad the training pairs to ensure equal sequence lengths within a batch
    padded_training_pairs = []
    for pair in training_pairs:
        padded_input = pad_sequence(pair[0], max_length)
        padded_target = pad_sequence(pair[1], max_length)
        padded_training_pairs.append((padded_input, padded_target))

    data_loader = DataLoader(padded_training_pairs, batch_size=batch_size, shuffle=True)

    for i in tqdm.tnrange(1, n_iters + 1):
        for batch in data_loader:
            input_tensor = batch[0].to(device)
            target_tensor = batch[1].to(device)

            loss = train(input_tensor, target_tensor, encoder,
                         decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss.item()
            plot_loss_total += loss.item()

        if i % print_every == 0:
            print_loss_avg = print_loss_total / (print_every * batch_size)
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, i / n_iters),
                                         i, i / n_iters * 100, print_loss_avg))

        if i % plot_every == 0:
            plot_loss_avg = plot_loss_total / (plot_every * batch_size)
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)


In [20]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)


In [21]:
# from torch.utils.data import DataLoader


# def pad_sequence(sequence, max_length):
#     # Pad sequence with zeros to match the max_length
#     padded_sequence = torch.zeros((max_length, *sequence[0].shape), dtype=sequence[0].dtype)
#     for i, tensor in enumerate(sequence):
#         padded_sequence[i] = tensor
#     return padded_sequence

# def trainIters(encoder, decoder, n_iters, batch_size=1, print_every=1000, plot_every=100, learning_rate=0.01):
#     global pairs  # Access the global pairs variable
#     start = time.time()
#     plot_losses = []
#     print_loss_total = 0  # Reset every print_every
#     plot_loss_total = 0  # Reset every plot_every

#     encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
#     decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
#     training_pairs = [tensorsFromPair(random.choice(pairs))
#                       for i in range(n_iters)]
#     criterion = nn.NLLLoss()

#     # Determine the maximum sequence length in the training pairs
#     max_length = max(max(len(pair[0]), len(pair[1])) for pair in training_pairs)

#     # Pad the training pairs to ensure equal sequence lengths within a batch
#     padded_training_pairs = []
#     for pair in training_pairs:
#         padded_input = pad_sequence(pair[0], max_length)
#         padded_target = pad_sequence(pair[1], max_length)
#         padded_training_pairs.append((padded_input, padded_target))

#     data_loader = DataLoader(padded_training_pairs, batch_size=batch_size, shuffle=True)

#     for i in tqdm.notebook.tnrange(1, n_iters + 1):
#         for batch in data_loader:
#             input_tensor = batch[0].to(device)
#             target_tensor = batch[1].to(device)

#             loss = train(input_tensor, target_tensor, encoder,
#                          decoder, encoder_optimizer, decoder_optimizer, criterion)
#             print_loss_total += loss.item()
#             plot_loss_total += loss.item()

#         if i % print_every == 0:
#             print_loss_avg = print_loss_total / (print_every * batch_size)
#             print_loss_total = 0
#             print('%s (%d %d%%) %.4f' % (timeSince(start, i / n_iters),
#                                          i, i / n_iters * 100, print_loss_avg))

#         if i % plot_every == 0:
#             plot_loss_avg = plot_loss_total / (plot_every * batch_size)
#             plot_losses.append(plot_loss_avg)
#             plot_loss_total = 0

#     showPlot(plot_losses)

In [22]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [23]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [32]:
hidden_size = 512
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 30000, batch_size=2, print_every=4000, plot_every=100)

  0%|          | 0/30000 [00:00<?, ?it/s]

RuntimeError: Expected hidden size (1, 1, 512), got [1, 15, 512]